# Initialisiere Spark Session
In diesem Notebook wird die Spark Session vorbereitet.

In [7]:
# Importieren der benötigten Bibliotheken
from pyspark.sql import SparkSession

# Beispiel für das Hinzufügen des GCS Connectors zu einer lokalen Spark-Session
spark = SparkSession.builder \
    .appName("Datatransformation") \
    .config('spark.sql.debug.maxToStringFields', '1000') \
    .config("spark.jars.packages", "com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.2") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Setup/prototyp-etl-pipline-d6cbb438aa70.json") \
    .getOrCreate()
    
# Überprüfen der SparkSession
spark

# Konfiguration der Google Cloud und Herunterladen der Paquet-Datei

In diesem Notebook wird die Google Cloud konfiguriert und eine Parque-Datei heruntergeladen.

In [8]:
import os
from google.cloud import storage
from google.oauth2 import service_account

# Dienstkonto-Datei laden
service_account_json = '/Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Setup/prototyp-etl-pipline-d6cbb438aa70.json'
credentials = service_account.Credentials.from_service_account_file(service_account_json)

# Google Cloud Storage Client initialisieren
client = storage.Client(credentials=credentials, project='prototyp-etl-pipline')

# Funktion zum Herunterladen eines Verzeichnisses von Google Cloud Storage
def download_directory_from_gcs(bucket_name, source_directory, local_directory):
    bucket = client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=source_directory)
    
    for blob in blobs:
        if not blob.name.endswith('/'):
            local_path = os.path.join(local_directory, os.path.relpath(blob.name, source_directory))
            os.makedirs(os.path.dirname(local_path), exist_ok=True)
            blob.download_to_filename(local_path)
            print(f"Datei {blob.name} erfolgreich heruntergeladen zu {local_path}.")

# Beispielaufruf der Funktion
bucket_name = 'prod_prototype'
source_directory = 'silver/sensitive/'
local_directory = '/tmp/applicant_data_cleaned/'

download_directory_from_gcs(bucket_name, source_directory, local_directory)


Datei silver/sensitive/._SUCCESS.crc erfolgreich heruntergeladen zu /tmp/applicant_data_cleaned/._SUCCESS.crc.
Datei silver/sensitive/.part-00000-ce17f809-34e6-4001-8576-06672977b23f-c000.snappy.parquet.crc erfolgreich heruntergeladen zu /tmp/applicant_data_cleaned/.part-00000-ce17f809-34e6-4001-8576-06672977b23f-c000.snappy.parquet.crc.
Datei silver/sensitive/_SUCCESS erfolgreich heruntergeladen zu /tmp/applicant_data_cleaned/_SUCCESS.
Datei silver/sensitive/part-00000-ce17f809-34e6-4001-8576-06672977b23f-c000.snappy.parquet erfolgreich heruntergeladen zu /tmp/applicant_data_cleaned/part-00000-ce17f809-34e6-4001-8576-06672977b23f-c000.snappy.parquet.


# Parquet Dateien in Dataframe laden

Hier werden die heruntergeladenen parquet Dateien wieder zusammengeführt und in Dataframe gespeichert.

In [9]:

# Parquet-Dateien laden
df_cleaned = spark.read.parquet(local_directory)
df_cleaned.show()

+--------------------+--------------------+---------------+----------------+--------------------+-------------------------+----------------------+--------------------+-----------------+-----------------+------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+------------+--------------------+-------------------+----------------------+---------+----+
|     Ablehnungsgrund|             Adresse|Bewerbungsdatum|Bewerbungsquelle|Bewertung Assessment|Bewertung Erstes Gespräch|Bewertung Prescreening|              E-Mail|Effektiver Gehalt|Einstellungsdatum|Geburtsdatum|Gehaltsvorstellungen|Geschlecht|    Headhunter Firma|     Headhunter Name|         Interviewer|           Job Titel|       Kandidaten-ID|   Nachname|Standort|      Status|           StellenID|      Telefonnummer|Veröffentlichungsdatum|  Vorname|  id|
+--------------------+--------------------+---------------+-------

In [ ]:
from pyspark.sql.functions import lit, concat_ws, monotonically_increasing_id

# Funktion zur Pseudonymisierung einer Spalte mit einer fortlaufenden Nummer
def pseudonymize_column(df, column_name, alias_name):
    """
    Pseudonymisiert eine Spalte in einem DataFrame mit einer fortlaufenden Nummer.
    
    Args:
        df (DataFrame): Das DataFrame, das pseudonymisiert werden soll.
        column_name (str): Der Name der Spalte, die pseudonymisiert werden soll.
        alias_name (str): Der Alias-Name für die pseudonymisierte Spalte.
        
    Returns:

In [27]:
from pyspark.sql.functions import lit, concat_ws, monotonically_increasing_id, regexp_replace, col

# Pseudonymisierung einer Spalte
def pseudonymize_column(df, column_name, alias_name):
    # Add your pseudonymization logic here
    return df.withColumn(alias_name, concat_ws("_", lit("pseudonymized_"+ column_name), monotonically_increasing_id()))


# Pseudonymisiere die Spalten
df_pseudonymized = pseudonymize_column(df, "Vorname", "Vorname")
df_pseudonymized = pseudonymize_column(df_pseudonymized, "Nachname", "Nachname")
df_pseudonymized = pseudonymize_column(df_pseudonymized, "E-Mail", "E-Mail")
df_pseudonymized = pseudonymize_column(df_pseudonymized, "Telefonnummer", "Telefonnummer")

# Pseudonymisiere Adresse: Ersetze Strassennamen und Nummern durch "Pseudonymized"
df_pseudonymized = df_pseudonymized.withColumn("Adresse", regexp_replace(col("Adresse"), r"^.*?(?=,)", "Pseudonymized"))

df_pseudonymized.show()

# Überprüfe, ob die Datei bereits existiert

df_pseudonymized.write.csv(output_path, header=True, mode="overwrite")

+--------------------+--------------------+---------------+----------------+--------------------+-------------------------+----------------------+--------------------+-----------------+-----------------+------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+------------+--------------------+--------------------+----------------------+--------------------+----+
|     Ablehnungsgrund|             Adresse|Bewerbungsdatum|Bewerbungsquelle|Bewertung Assessment|Bewertung Erstes Gespräch|Bewertung Prescreening|              E-Mail|Effektiver Gehalt|Einstellungsdatum|Geburtsdatum|Gehaltsvorstellungen|Geschlecht|    Headhunter Firma|     Headhunter Name|         Interviewer|           Job Titel|       Kandidaten-ID|            Nachname|Standort|      Status|           StellenID|       Telefonnummer|Veröffentlichungsdatum|             Vorname|  id|
+--------------------+--

# Wieder auf Google hochladen